In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../TextFiles/smsspamcollection.tsv','\t')

In [3]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [4]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [5]:
df.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split


In [7]:
X = df.message

In [8]:
y=df.label

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
cv = CountVectorizer()

In [15]:
# Fit the vectorizer to the data (build a vocab, count the number of words...)
#cv.fit(X_train)
#X_train_counts = cv.transform(X_train)
#Transform the original text message to a Vector

X_train_counts = cv.fit_transform(X_train)

In [16]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [17]:
X_train.shape

(3733,)

In [18]:
X_train_counts.shape

(3733, 7082)

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer

In [20]:
tfidf_transf = TfidfTransformer()

In [21]:
X_train_tfidf = tfidf_transf.fit_transform(X_train_counts)

In [22]:
X_train_tfidf.shape

(3733, 7082)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
tfidf_vectorizer = TfidfVectorizer()

In [26]:
X_train_tfidf_vect = tfidf_vectorizer.fit_transform(X_train)

In [27]:
from sklearn.svm import LinearSVC

In [28]:
clf = LinearSVC()

In [29]:
clf.fit(X_train_tfidf_vect,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [31]:
from sklearn.pipeline import Pipeline

In [32]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [33]:
text_clf.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [34]:
predictions = text_clf.predict(X_test)

In [35]:
X_test

3245    Squeeeeeze!! This is christmas hug.. If u lik ...
944     And also I've sorta blown him off a couple tim...
1044    Mmm thats better now i got a roast down me! i...
2484        Mm have some kanji dont eat anything heavy ok
812     So there's a ring that comes with the guys cos...
2973    Sary just need Tim in the bollox &it hurt him ...
2991    Love isn't a decision, it's a feeling. If we c...
2942    My supervisor find 4 me one lor i thk his stud...
230                    Dear good morning now only i am up
1181                           I'm in chennai velachery:)
1912    Lol grr my mom is taking forever with my presc...
1992    No other Valentines huh? The proof is on your ...
5435                    I'm wif him now buying tix lar...
4805    Er, hello, things didn‘t quite go to plan – is...
401     FREE RINGTONE text FIRST to 87131 for a poly o...
1859                     Sir, i am waiting for your call.
1344    Crazy ar he's married. Ü like gd looking guys ...
2952          

In [36]:
from sklearn.metrics import confusion_matrix,classification_report

In [37]:
print(confusion_matrix(y_test,predictions))

[[1586    7]
 [  12  234]]


In [38]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

   micro avg       0.99      0.99      0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [39]:
from sklearn import metrics

In [40]:
metrics.accuracy_score(y_test,predictions)

0.989668297988037

In [42]:
text_clf.predict(["Him how are you today"])

array(['ham'], dtype=object)

In [43]:
text_clf.predict(["totally free for you"])

array(['ham'], dtype=object)

In [44]:
text_clf.predict(["click inside for promotions"])

array(['ham'], dtype=object)

In [48]:
text_clf.predict(["TEXT FREE TO THIS NUMBER "])

array(['spam'], dtype=object)